In [1]:
import os
import keyring
import glob
from ftplib import FTP
from arcgis import GIS

# Toggles

In [2]:
# Switch to false to disable
update_maps = True
update_charts = True

# Update files in arcgis online

In [3]:
# Sign into ArcGIS Online
un = 'analytics_wfrc'
pw = keyring.get_password('Analytics AGOL', un)
gis = GIS(username=un, password=pw)

In [4]:
# store path to the new data
cityarea_zip = r".\Outputs\map\CITYAREA.gdb.zip"
county_zip = r".\Outputs\map\COUNTY.gdb.zip"
distmed_zip = r".\Outputs\map\DISTMED.gdb.zip"
otherarea_zip = r".\Outputs\map\OTHERAREA.gdb.zip"
taz_zip = r".\Outputs\map\TAZ.gdb.zip"

In [5]:
# store item ids
cityarea_item_id = '265bfbe2cae7490196e230eced9109b2'
county_item_id = 'cb97017428fe470280fd015b4d7d61bf'
distmed_item_id = 'bdf485e1e5e94c16b08b2464bc4f50a5'
otherarea_item_id = '56ae4b1483e54eff932c24127d496f46'
taz_item_id = '844f78208e604aabb0d14d5ae827712d'

In [6]:
def update_and_republish_item(id, new_zip):
    item = gis.content.get(id)
    name = item['title']

    # update the data source
    print(f'Updating data source for {name}...')
    item.update(data=new_zip) # store this result and test, while loop?

    # republish the feature layer(s)
    print(f'Publishing {name}...')
    status = 0
    attempts = 0
    attempts_allowed = 15

    while status == 0:
        if attempts < attempts_allowed:
            attempts = attempts + 1
            try:
                item.publish(overwrite=True)
            except:
                print(f'--Failed to publish {name} after {attempts} attempts. Trying again...')
            else:
                status = 1
                print(f'--Successfully published {name} after {attempts} attempt(s)!')             
        else:
            status = 2
            raise Exception(f'--Error: Failed to publish {name} after {attempts} attempts!')


In [7]:
if update_maps == True:
    
    # update the data source, then republish
    update_and_republish_item(cityarea_item_id, cityarea_zip)
    update_and_republish_item(county_item_id, county_zip)
    update_and_republish_item(distmed_item_id, distmed_zip)
    update_and_republish_item(otherarea_item_id, otherarea_zip)
    update_and_republish_item(taz_item_id, taz_zip)

Updating data source for CITYAREA_gdb...
Publishing CITYAREA_gdb...
--Successfully published CITYAREA_gdb after 1 attempt(s)!
Updating data source for COUNTY_gdb...
Publishing COUNTY_gdb...
--Successfully published COUNTY_gdb after 1 attempt(s)!
Updating data source for DISTMED_gdb...
Publishing DISTMED_gdb...
--Successfully published DISTMED_gdb after 1 attempt(s)!
Updating data source for OTHERAREA_gdb...
Publishing OTHERAREA_gdb...
--Successfully published OTHERAREA_gdb after 1 attempt(s)!
Updating data source for TAZ_gdb...
Publishing TAZ_gdb...
--Successfully published TAZ_gdb after 1 attempt(s)!


# Update files on server

In [10]:
local_chart_folder = r'.\Outputs\chart'
un2= 'wfrcorg'
pw2 = keyring.get_password('WFRC File Server', un2)
ip = keyring.get_password('WFRC File Server IP', un2)
ftp = FTP(ip, timeout=1200)
ftp.login(user=un2, passwd =pw2)

'230 OK. Current restricted directory is /'

In [11]:
if update_charts == True:
    geogs = ['CITYAREA', 'COUNTY', 'DISTMED', 'OTHERAREA', 'TAZ']

    for geog in geogs:
        ftp.cwd(f'/public_html/household-job-forecast-map/widgets/SESidebar/data/chart/{geog}')
        
        old_jsons = [item for item in ftp.nlst() if item not in ['.','..'] and item.endswith('.json')]
        if len(old_jsons) > 0:
            print(f'deleting old jsons ({geog})...')
            for json in old_jsons:
                # print(json)
                ftp.delete(json)
        else:
            print(f'no jsons to delete ({geog})...')

        print(f'uploading new jsons ({geog})...')
        new_jsons = glob.glob(os.path.join(local_chart_folder, geog, '*.json'))
        
        for json in new_jsons:
            #print(json)
            with open(json,'rb') as file:
                ftp.storbinary('STOR ' + os.path.basename(json), file)

deleting old jsons (CITYAREA)...
uploading new jsons (CITYAREA)...
deleting old jsons (COUNTY)...
uploading new jsons (COUNTY)...
deleting old jsons (DISTMED)...
uploading new jsons (DISTMED)...
deleting old jsons (OTHERAREA)...
uploading new jsons (OTHERAREA)...
deleting old jsons (TAZ)...
uploading new jsons (TAZ)...


# Update Downloable dataset on server

In [12]:
ftp.cwd(f'/public_html/MapsData/SocioEconomicData')
ftp.delete('WFRC_SE_FORECAST_2019_2050_DRAFT.zip')

zip_for_download = r".\Outputs\WFRC_SE_FORECAST_2019_2050_DRAFT.zip"
with open(zip_for_download,'rb') as file:
                ftp.storbinary('STOR ' + os.path.basename(zip_for_download), file)

In [13]:
# close the connection
ftp.quit()

'221-Goodbye. You uploaded 73508 and downloaded 0 kbytes.\n221 Logout.'